### 練習問題2.81
Louis Reasonerは、引数の型がすでに同じであっても、apply-generic は引数をお互いの型に強制型変換しようとしてもいいのではないかと気がついた。<br />
そのため、それぞれの型の引数をそれ⾃⾝の型に強制型変換 (coerce) する⼿続きを強制型変換テーブルに⼊れる必要があると彼は考えた。<br />
例えば、上に⽰した scheme-number->complex という強制型変換に加え、彼は次のことを⾏う。</br>

    (define (scheme-number->scheme-number n) n)
    (define (complex->complex z) z)
    (put-coercion 'scheme-number 'scheme-number scheme-number->scheme-number)
    (put-coercion 'complex 'complex complex->complex)

In [69]:
(define (assoc key records)
  (cond ((null? records) #f)
        ((equal? key (caar records)) (car records))
        (else (assoc key (cdr records)))
        )
  )

(define (lookup key-1 key-2 table)
  (let ((subtable (assoc key-1 (cdr table))))
    (if subtable
        (let ((record (assoc key-2 (cdr subtable))))
          (if record (cdr record)
              #f
              )
          )
        #f
        )
    )
  )

(define (insert! key-1 key-2 value table)
  (let ((subtable (assoc key-1 (cdr table))))
    (if subtable
        (let ((record (assoc key-2 (cdr subtable))))
          (if record (set-cdr! record value)
              (set-cdr! subtable (cons (cons key-2 value) (cdr subtable)))
              )
          )
        (set-cdr! table (cons (list key-1 (cons key-2 value)) (cdr table)))
        )
    )
  'ok)

(define (make-table)
  (let ((local-table (list '*table*)))
    (define (lookup key-1 key-2)
      (let ((subtable (assoc key-1 (cdr local-table))))
        (if subtable
            (let ((record (assoc key-2 (cdr subtable))))
              (if record (cdr record) #f))
            #f)
        )
      )
    (define (insert! key-1 key-2 value)
      (let ((subtable (assoc key-1 (cdr local-table))))
        (if subtable
            (let ((record (assoc key-2 (cdr subtable))))
              (if record (set-cdr! record value)
                  (set-cdr! subtable (cons (cons key-2 value) (cdr subtable)))
                  )
              )
            (set-cdr! local-table (cons (list key-1 (cons key-2 value)) (cdr local-table )))
            )
        )
      'ok)
    (define (dispatch m)
      (cond ((eq? m 'lookup-proc) lookup)
            ((eq? m 'insert-proc!) insert!)
            (else (error "Unknown operation: TABLE" m)))
      )
    dispatch)
  )

(define operation-table (make-table))
(define get (operation-table 'lookup-proc))
(define put (operation-table 'insert-proc!))
(define get-coercion get)
(define put-coercion put)

; タグつきデータのコンストラクタ
(define (attach-tag type-tag contents)
  (cons type-tag contents))
; タグを返すセレクタ
(define (type-tag datum)
  (if (pair? datum) (car datum)
      (error "Bad tagged datum: TYPE-TAG" datum))
  )
; データを返すセレクタ
(define (contents datum)
  (if (pair? datum) (cdr datum)
      (error "Bad tagged datum: CONTENTS" datum))
  )

;
(define (apply-generic op . args)
  (let ((type-tags (map type-tag args)))
    (let ((proc (get op type-tags)))
      (if proc
          (apply proc (map contents args))
          (if (= (length args) 2)
              (let ((type1 (car type-tags))
                    (type2 (cadr type-tags))
                    (a1 (car args))
                    (a2 (cadr args)))
                (let ((t1->t2 (get-coercion type1 type2))
                      (t2->t1 (get-coercion type2 type1)))
                  (cond (t1->t2
                         (apply-generic op (t1->t2 a1) a2))
                        (t2->t1
                         (apply-generic op a1 (t2->t1 a2)))
                        (else (error "No method for these types"
                                     (list op type-tags ))))))
              (error "No method for these types"
                     (list op type-tags )))))))

In [70]:
(define (add x y) (apply-generic 'add x y))
(define (sub x y) (apply-generic 'sub x y))
(define (mul x y) (apply-generic 'mul x y))
(define (div x y) (apply-generic 'div x y))

In [71]:
(define (install-scheme-number-package)
  (define (tag x) (attach-tag 'scheme-number x))
  (put 'add '(scheme-number scheme-number) (lambda (x y) (tag (+ x y))))
  (put 'sub '(scheme-number scheme-number) (lambda (x y) (tag (- x y))))
  (put 'mul '(scheme-number scheme-number) (lambda (x y) (tag (* x y))))
  (put 'div '(scheme-number scheme-number) (lambda (x y) (tag (/ x y))))
  (put 'make 'scheme-number (lambda (x) (tag x)))
  (put 'exp '(scheme-number scheme-number) (lambda (x y) (tag (expt x y))))
'done)

(define (make-scheme-number n) ((get 'make 'scheme-number) n))

(install-scheme-number-package)

done

In [72]:
(define (gcd a b)
  (if (= b 0) a
      (gcd b (remainder a b)))
  )

(define (install-rational-package)
  ;; 内部⼿続き
  (define (numer x) (car x))
  (define (denom x) (cdr x))
  (define (make-rat n d)
    (let ((g (gcd n d))) (cons (/ n g) (/ d g))))
  (define (add-rat x y) (make-rat (+ (* (numer x) (denom y))
                                     (* (numer y) (denom x)))
                                  (* (denom x) (denom y))))
  (define (sub-rat x y) (make-rat (- (* (numer x) (denom y))
                                     (* (numer y) (denom x)))
                                  (* (denom x) (denom y))))
  (define (mul-rat x y) (make-rat (* (numer x) (numer y))
                                  (* (denom x) (denom y))))
  (define (div-rat x y) (make-rat (* (numer x) (denom y)) (* (denom x) (numer y))))
  ;; システムのほかの部分とのインターフェイス
  (define (tag x) (attach-tag 'rational x))
  (put 'add '(rational rational) (lambda (x y) (tag (add-rat x y))))
  (put 'sub '(rational rational) (lambda (x y) (tag (sub-rat x y))))
  (put 'mul '(rational rational) (lambda (x y) (tag (mul-rat x y)))) 
  (put 'div '(rational rational) (lambda (x y) (tag (div-rat x y))))
  (put 'make 'rational (lambda (n d) (tag (make-rat n d))))
  'done)

(define (make-rational n d) ((get 'make 'rational) n d))

(install-rational-package)

done

In [73]:
(import "math")
(define (make-from-mag-ang r a)
  (define (dispatch op)
    (cond ((eq? op 'real-part) (* r (math.cos a)))
          ((eq? op 'imag-part) (* r (math.sin a)))
          ((eq? op 'magnitude) r)
          ((eq? op 'angle) a)
          (else (error "Unknown op: MAKE-FROM-MAG-ANG" op))
          )
  )
dispatch)

(define (square x)(* x x))

(define (install-rectangular-package)
  ;; 内部⼿続き
  (define (real-part z) (car z))
  (define (imag-part z) (cdr z))
  (define (make-from-real-imag x y) (cons x y))
  (define (magnitude z) (sqrt (+ (square (real-part z)) (square (imag-part z)))))
  (define (angle z) (math.atan2 (imag-part z) (real-part z)))
  (define (make-from-mag-ang r a) (cons (* r (math.cos a)) (* r (math.sin a))))
  ;; システムのほかの部分とのインターフェイス
  (define (tag x) (attach-tag 'rectangular x))
  (put 'real-part '(rectangular) real-part)
  (put 'imag-part '(rectangular) imag-part)
  (put 'magnitude '(rectangular) magnitude)
  (put 'angle '(rectangular) angle)
  (put 'make-from-real-imag 'rectangular (lambda (x y) (tag (make-from-real-imag x y))))
  (put 'make-from-mag-ang 'rectangular (lambda (r a) (tag (make-from-mag-ang r a))))
'done)

(define (install-polar-package)
  ;; 内部⼿続き
  (define (magnitude z) (car z))
  (define (angle z) (cdr z))
  (define (make-from-mag-ang r a) (cons r a))
  (define (real-part z) (* (magnitude z) (math.cos (angle z))))
  (define (imag-part z) (* (magnitude z) (math.sin (angle z))))
  (define (make-from-real-imag x y) (cons (sqrt (+ (square x) (square y))) (math.atan2 y x)))
  ;; システムのほかの部分とのインターフェイス
  (define (tag x) (attach-tag 'polar x))
  (put 'real-part '(polar) real-part)
  (put 'imag-part '(polar) imag-part)
  (put 'magnitude '(polar) magnitude)
  (put 'angle '(polar) angle)
  (put 'make-from-real-imag 'polar (lambda (x y) (tag (make-from-real-imag x y))))
  (put 'make-from-mag-ang 'polar (lambda (r a) (tag (make-from-mag-ang r a))))
  'done)

(define (real-part z) (apply-generic 'real-part z))
(define (imag-part z) (apply-generic 'imag-part z))
(define (magnitude z) (apply-generic 'magnitude z))
(define (angle z) (apply-generic 'angle z))

(define (make-from-real-imag x y) ((get 'make-from-real-imag 'rectangular) x y))
(define (make-from-mag-ang r a) ((get 'make-from-mag-ang 'polar) r a))

(install-rectangular-package)
(install-polar-package)

done

In [74]:
(define (install-complex-package)
  ;; 直交形式パッケージと極形式パッケージからインポートした⼿続き
  (define (make-from-real-imag x y) ((get 'make-from-real-imag 'rectangular) x y))
  (define (make-from-mag-ang r a) ((get 'make-from-mag-ang 'polar) r a))
  ;; 内部⼿続き
  (define (add-complex z1 z2) (make-from-real-imag (+ (real-part z1) (real-part z2)) (+ (imag-part z1) (imag-part z2))))
  (define (sub-complex z1 z2) (make-from-real-imag (- (real-part z1) (real-part z2)) (- (imag-part z1) (imag-part z2))))
  (define (mul-complex z1 z2) (make-from-mag-ang (* (magnitude z1) (magnitude z2)) (+ (angle z1) (angle z2))))
  (define (div-complex z1 z2) (make-from-mag-ang (/ (magnitude z1) (magnitude z2)) (- (angle z1) (angle z2))))
  ;; システムのほかの部分とのインターフェイス
  (define (tag z) (attach-tag 'complex z))
  (put 'add '(complex complex) (lambda (z1 z2) (tag (add-complex z1 z2))))
  (put 'sub '(complex complex) (lambda (z1 z2) (tag (sub-complex z1 z2))))
  (put 'mul '(complex complex) (lambda (z1 z2) (tag (mul-complex z1 z2))))
  (put 'div '(complex complex) (lambda (z1 z2) (tag (div-complex z1 z2))))
  (put 'make-from-real-imag 'complex (lambda (x y) (tag (make-from-real-imag x y))))
  (put 'make-from-mag-ang 'complex (lambda (r a) (tag (make-from-mag-ang r a))))
  'done)

(define (make-complex-from-real-imag x y) ((get 'make-from-real-imag 'complex) x y))
(define (make-complex-from-mag-ang r a) ((get 'make-from-mag-ang 'complex) r a))

(install-complex-package)

done

a. Louis の強制型変換⼿続きを組み込むと、もし scheme-number型の⼆つの引数や complex 型の⼆つの引数とある演算に対して apply-generic が呼ばれ、その演算がテーブル内でそれらの型に対して⾒つからない場合は、何が起こるだろうか。<br />
例えば、ジェネリックな指数関数演算を定義したとする。

    (define (exp x y) (apply-generic 'exp x y))

そして、Scheme-number パッケージに対して指数関数⼿続きを追加し、ほかのパッケージには追加しないとする。

    ;; 以下は Scheme-number パッケージに追加する
    (put 'exp '(scheme-number scheme-number)
    (lambda (x y) (tag (expt x y))))
    ; 基本⼿続き expt を使う

exp を⼆つの複素数の引数で呼び出した場合、何が起こるだろうか。

In [57]:
(define (scheme-number->complex n)
(make-complex-from-real-imag (contents n) 0))
(put-coercion 'scheme-number 'complex scheme-number->complex)
(define (scheme-number->scheme-number n) n)
(define (complex->complex z) z)
(put-coercion 'scheme-number 'scheme-number scheme-number->scheme-number)
(put-coercion 'complex 'complex complex->complex)

ok

In [58]:
(define (exp x y) (apply-generic 'exp x y))


In [59]:
(exp (make-scheme-number 5) (make-scheme-number 2))

(scheme-number . 25.0)

In [60]:
(exp (make-scheme-number 2) (make-scheme-number 5))

(scheme-number . 32.0)

In [61]:
(define z1 (make-complex-from-real-imag 1 3))

In [62]:
(define z2 (make-complex-from-real-imag 2 4))

In [63]:
(exp z1 z2)


Traceback (most recent call last):
  File "In [63]", line 1, col 1, in 'exp'
  File "In [58]", line 1, col 19, in 'apply-generic'
  File "In [51]", line 86, col 3, in 'let'
  File "In [51]", line 87, col 5, in 'let'
  File "In [51]", line 91, col 15, in 'let'
  File "In [51]", line 95, col 17, in 'let'
  File "In [51]", line 98, col 26, in 'apply-generic'
  File "In [51]", line 86, col 3, in 'let'
  File "In [51]", line 87, col 5, in 'let'
  File "In [51]", line 91, col 15, in 'let'
  File "In [51]", line 95, col 17, in 'let'
  File "In [51]", line 98, col 26, in 'apply-generic'
  File "In [51]", line 86, col 3, in 'let'
  File "In [51]", line 87, col 5, in 'let'
  File "In [51]", line 91, col 15, in 'let'
  File "In [51]", line 95, col 17, in 'let'
  File "In [51]", line 98, col 26, in 'apply-generic'
  File "In [51]", line 86, col 3, in 'let'
  File "In [51]", line 87, col 5, in 'let'
  File "In [51]", line 91, col 15, in 'let'
  File "In [51]", line 95, col 17, in 'let'
  File "In 

【aの答え】<br />
complex型に対してexpを実行すると、無限ループになる。

b. 同じ型の引数に対する強制型変換について⼿を加えないといけないとする Louis の考え⽅は正しいだろうか。<br />
それとも、apply-generic はそのままの状態で正しく動作するだろうか。

【bの答え】<br />
正しくない。
外部で

    (put-coercion 'complex 'complex complex->complex)
    
を実行しなければ、apply-genericはそのままで問題ないが、<br />
外部で

    (put-coercion 'complex 'complex complex->complex)
    
が実行されることを想定すると、
apply-genericは修正すべきと考える。

c. apply-generic を修正し、⼆つの引数が同じ型であれば強制型変換を試⾏しないようにせよ。

In [65]:
(define (apply-generic op . args)
  (let ((type-tags (map type-tag args)))
    (let ((proc (get op type-tags)))
      (if proc
          (apply proc (map contents args))
          (if (= (length args) 2)
              (let ((type1 (car type-tags))
                    (type2 (cadr type-tags))
                    (a1 (car args))
                    (a2 (cadr args)))
                (if (not (eq? type1 type2))
                    (let ((t1->t2 (get-coercion type1 type2))
                          (t2->t1 (get-coercion type2 type1)))
                      (cond (t1->t2
                             (apply-generic op (t1->t2 a1) a2))
                            (t2->t1
                             (apply-generic op a1 (t2->t1 a2)))
                            (else (error "No method for these types" (list op type-tags )))
                            )
                      )
                    (error "Same types" (list op type-tags ))
                    )
                )
              (error "No method for these types" (list op type-tags )))))
      )
    )

In [66]:
(exp z1 z2)


Traceback (most recent call last):
  File "In [66]", line 1, col 1, in 'exp'
  File "In [58]", line 1, col 19, in 'apply-generic'
  File "In [65]", line 2, col 3, in 'let'
  File "In [65]", line 3, col 5, in 'let'
  File "In [65]", line 7, col 15, in 'let'
  File "In [65]", line 21, col 21, in 'error'
UnhandledException: string argument expected, got 'Symbol'



In [67]:
(exp (make-scheme-number 5) (make-scheme-number 2))

(scheme-number . 25.0)

In [68]:
(exp (make-scheme-number 2) (make-scheme-number 5))

(scheme-number . 32.0)